<a href="https://colab.research.google.com/github/IMC-Croissant/AlgoTrading/blob/main/shared_data_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The purpose of this is to get a sense of how the first trading day data looks like
We will evaluate an algo trading run and see how each product changes in price over time

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Load the csv file from Google Drive into a pandas dataframe
file_path = '/content/drive/My Drive/IMC/trail.csv'
df_trade = pd.read_csv(file_path,sep=';')

# Display the first 5 rows of the dataframe
print(df_trade.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   day  timestamp  product  bid_price_1  bid_volume_1  bid_price_2  \
0   -1          0   PEARLS         9998             1       9995.0   
1   -1          0  BANANAS         4893             1       4892.0   
2   -1        100   PEARLS         9996             1       9995.0   
3   -1        100  BANANAS         4892            31          NaN   
4   -1        200  BANANAS         4892            22          NaN   

   bid_volume_2  bid_price_3  bid_volume_3  ask_price_1  ask_volume_1  \
0          30.0          NaN           NaN        10005            30   
1          30.0          NaN           NaN         4899            30   
2          30.0          NaN           NaN        10002             6   
3           NaN          NaN           NaN         4896             7   
4           NaN          NaN           NaN         4899            22   

   ask_pric

In [ ]:
df_trade['mid_price'] = (df_trade['bid_price_1'] + df_trade['ask_price_1']) / 2
df_trade

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,PEARLS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,-1,0,BANANAS,4893,1,4892.0,30.0,NaN,NaN,4899,30,NaN,NaN,NaN,NaN,4896.0,0.0
2,-1,100,PEARLS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,-1,100,BANANAS,4892,31,NaN,NaN,NaN,NaN,4896,7,4898.0,31.0,NaN,NaN,4894.0,0.0
4,-1,200,BANANAS,4892,22,NaN,NaN,NaN,NaN,4899,22,NaN,NaN,NaN,NaN,4895.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-1,199700,PEARLS,10002,1,9996.0,2.0,9995.0,28.0,10004,2,10005.0,28.0,NaN,NaN,10003.0,0.0
3996,-1,199800,BANANAS,4905,28,NaN,NaN,NaN,NaN,4911,28,NaN,NaN,NaN,NaN,4908.0,0.0
3997,-1,199800,PEARLS,9996,1,9995.0,27.0,NaN,NaN,10004,1,10005.0,27.0,NaN,NaN,10000.0,0.0
3998,-1,199900,BANANAS,4906,6,4905.0,25.0,NaN,NaN,4912,25,NaN,NaN,NaN,NaN,4909.0,0.0


## Basic information about the data, variances, etc

In [ ]:
df_trade.groupby('product').agg({'mid_price': ['mean', 'std', 'min', 'max']})

mid_price                           
               mean       std     min      max
product                                       
BANANAS  4898.08675  5.318552  4883.0   4911.0
PEARLS   9999.99350  1.493427  9996.5  10003.5

Overall, standard deviation in Bananas higher than pearls.  Pearls has a range of ~6 while Bananas has a range of 28.  This is a good indicator that Bananas is more volatile than Pearls.  Perhaps swing trading on bananas would be a good strategy, while a regular market making strategy on pearls would be a good strategy.

## Evaluate products over timestamp and provide midprice between bid_price_1 and ask_price_1

In [ ]:
df_viz = df_trade.groupby(['timestamp', 'product'])['mid_price'].mean().reset_index()
fig = px.line(df_viz, x='timestamp', y='mid_price', facet_row='product')
fig.update_yaxes(matches=None)
fig.show()

In [ ]:
# add some columns for later analyses
df_trade["bid_diff"] = np.round(df_trade.bid_price_1.diff().fillna(0), 2)
df_trade["ask_diff"] = np.round(df_trade.ask_price_1.diff().fillna(0), 2)
df_trade["ba_spread"] = df_trade.ask_price_1 - df_trade.bid_price_1
df_trade["ba_spread_diff"] = np.round(df_trade.ba_spread.diff().fillna(0), 6)
df_trade["widen"] = df_trade.ba_spread_diff > 0
df_trade["tighten"] = df_trade.ba_spread_diff < 0
df_trade["widen_both_side"] = (df_trade.bid_diff < 0) & (df_trade.ask_diff > 0)
# print(df["widen_both_side"]
df_trade["tighten_both_side"] = (df_trade.bid_diff > 0) & (df_trade.ask_diff < 0)
# print(df["tighten_both_side"])
df_trade[df_trade.widen]

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,...,mid_price,profit_and_loss,bid_diff,ask_diff,ba_spread,ba_spread_diff,widen,tighten,widen_both_side,tighten_both_side
4,-1,200,BANANAS,4892,22,NaN,NaN,NaN,NaN,4899,...,4895.5,0.0,0.0,3.0,7,3.0,True,False,False,False
5,-1,200,PEARLS,9996,1,9995.0,21.0,NaN,NaN,10004,...,10000.0,0.0,5104.0,5105.0,8,1.0,True,False,False,False
7,-1,300,BANANAS,4893,25,NaN,NaN,NaN,NaN,4899,...,4896.0,0.0,-5103.0,-5099.0,6,4.0,True,False,False,False
11,-1,500,BANANAS,4893,24,NaN,NaN,NaN,NaN,4899,...,4896.0,0.0,-5109.0,-5105.0,6,4.0,True,False,False,False
13,-1,600,PEARLS,9996,2,9995.0,24.0,NaN,NaN,10002,...,9999.0,0.0,5104.0,5105.0,6,1.0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,-1,199500,BANANAS,4905,22,NaN,NaN,NaN,NaN,4912,...,4908.5,0.0,-5097.0,-5093.0,7,4.0,True,False,False,False
3992,-1,199600,PEARLS,9995,28,NaN,NaN,NaN,NaN,10005,...,10000.0,0.0,-1.0,7.0,10,8.0,True,False,True,False
3996,-1,199800,BANANAS,4905,28,NaN,NaN,NaN,NaN,4911,...,4908.0,0.0,-5097.0,-5093.0,6,4.0,True,False,False,False
3997,-1,199800,PEARLS,9996,1,9995.0,27.0,NaN,NaN,10004,...,10000.0,0.0,5091.0,5093.0,8,2.0,True,False,False,False


In [ ]:
# Calculate the bid-ask spread for each row
df_trade['spread'] = df_trade['ask_price_1'] - df_trade['bid_price_1']

# Calculate the mid price for each row
df_trade['mid_price'] = (df_trade['ask_price_1'] + df_trade['bid_price_1']) / 2

# Determine how many times the bid-ask spread widened
df_trade['spread_widened'] = np.round(df_trade['spread'].diff().fillna(0),2) > 0

# Determine how many times the bid-ask spread widened on both sides
df_trade['spread_widened_both'] = ((df_trade['bid_price_1'].diff() < 0) & (df_trade['ask_price_1'].diff() > 0))

# Determine how many times the bid-ask spread widened on one side
df_trade['spread_widened_one'] = (((df_trade['bid_price_1'].diff() == 0) & (df_trade['ask_price_1'].diff() > 0))) | (((df_trade['bid_price_1'].diff() < 0) & (df_trade['ask_price_1'].diff() == 0)))

# Determine how many times the bid-ask spread tightened
df_trade['spread_tightened'] = np.round(df_trade['spread'].diff().fillna(0),2) < 0

# Count the number of times the bid-ask spread widened and tightened
num_widened = df_trade['spread_widened'].sum()
num_widened_both = df_trade['spread_widened_both'].sum()
num_widened_one = df_trade['spread_widened_one'].sum()
num_tightened = df_trade['spread_tightened'].sum()

# Calculate the proportion of times the bid-ask spread widened on both sides
prop_widened_both = num_widened_both / num_widened

# Calculate the proportion of times the bid-ask spread widened on one side
prop_widened_one = num_widened_one / num_widened

# Print the results
print("Number of times the bid-ask spread widened:", num_widened)
print("Proportion of times the bid-ask spread widened on both sides:", prop_widened_both)
print("Proportion of times the bid-ask spread widened on one side:", prop_widened_one)
print("Number of times the bid-ask spread tightened:", num_tightened,"\n")

Number of times the bid-ask spread widened: 1777
Proportion of times the bid-ask spread widened on both sides: 0.05346088913899831
Proportion of times the bid-ask spread widened on one side: 0.12042768711311198
Number of times the bid-ask spread tightened: 1780 

